In [2]:
!pip install pandas scikit-learn numpy



In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/tmdb-movie-metadata


In [4]:
import pandas as pd
import ast


# **Loading Dataset** 

In [5]:
df = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")
print(df.head())

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

# **Preprocessing** 

In [6]:
df = df[['title', 'overview', 'genres', 'keywords']].dropna()

In [7]:
import json
#parsing to getAZ 
def parse_json(text):
    if isinstance(text, str) and text.strip():
        try:
            json_list = json.loads(text)  # Convert string to list of dicts
            if isinstance(json_list, list):
                return ", ".join([d["name"] for d in json_list if "name" in d])  # Extract names safely
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {text} -> {e}")
            return ""
    return ""


In [8]:
df['genres'] = df['genres'].apply(parse_json)
df['keywords'] = df['keywords'].apply(parse_json)

In [9]:
# Combine text features into a single column
df['content'] = df['overview'] + " " + df['genres'] + " " + df['keywords']

In [10]:
df.head()


,title,overview,genres,keywords,content
0,Avatar,"In the 22nd century, a paraplegic Marine is di...","Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...,"Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...",Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca...","Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...","John Carter is a war-weary, former military ca..."


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text into TF-IDF vectors
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df["content"])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
#recommendation system using cosine similarity
def recommend_movies(query, top_n=5):
    query_vec = vectorizer.transform([query])
    similarity_scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    return df.iloc[top_indices][["title", "overview"]]

# Example Query
query = "I love thrilling action movies set in space, with a comedic twist."
recommendations = recommend_movies(query)
print(recommendations)


              title                                           overview
880      Grindhouse  Two full length feature horror movies written ...
2198        Lockout  Set in the near future, Lockout follows a fals...
1054  Scary Movie 2  While the original parodied slasher flicks lik...
1531      Moonraker  During the transportation of a Space Shuttle a...
0            Avatar  In the 22nd century, a paraplegic Marine is di...


In [15]:
# Example Query
query = "I like ghost movies."
recommendations = recommend_movies(query)
print(recommendations)


                     title                                           overview
880             Grindhouse  Two full length feature horror movies written ...
148           Ghostbusters  Following a ghost invasion of Manhattan, paran...
2359              Blow Out  Jack Terry is a master sound recordist who wor...
1054         Scary Movie 2  While the original parodied slasher flicks lik...
3000  Insidious: Chapter 3  A twisted new tale of terror begins for a teen...


In [ ]:
#Salary=$6400/month